In [1]:
import os
os.chdir('K:/ecole/github/televerser/wisdom of crowds')
import pandas as pd

In [2]:
targetyear=[2019,2020]

target_commodity_consensus=['ALUMINIUM',
 'BRENT OIL',
 'COPPER',
 'GOLD',
 'HENRY HUB',
 'NICKEL',
 'PLATINUM',
 'SILVER',
 'THERMAL COAL',
 'WTI OIL',
 'ZINC']

target_commodity_spot=['Aluminum $/t',
 'Brent Oil $/bbl',
 'Copper $/t',
 'Gold $/oz',
 'Nat Gas $/MMBtu',
 'Nickel $/t',
 'Platinum $/oz',
 'Silver $/oz',
 'Thermal Coal $/t',
 'WTI Oil $/bbl',
 'Zinc LME Cash $/t']

target_source=['BMO',  'CITIGROUP',
    'GOLDMAN SACHS', 'BOAML', 'JP MORGAN', 
    'TD SECURITIES', 'COMMONWEALTH BANK', 'DEUTSCHE BANK']

lbs2ton=2204.6226218

In [3]:
#get consensus
cons=pd.read_csv('consensus.csv')

#remove useless commodities
cons=cons[cons['Commodity Product Name'].isin(target_commodity_consensus)]

#datetime format
for i in ['Forecast Publish Date','Date',
          'First Publish Date','Date Of Update']:
    cons[i]=pd.to_datetime(cons[i])

#only include recent year
cons=cons[cons['Date'].dt.year.isin(targetyear)]

#only include certain banks
cons=cons[cons['Source Name'].isin(target_source)]

#take the latest date as the publish date
cons['publish date']=cons[['Forecast Publish Date',
    'First Publish Date','Date Of Update']].max(axis=1)

#only care about year and month
cons['publish date']=cons['publish date'].astype(str).apply(lambda x:x[:8]+'01')
cons['publish date']=pd.to_datetime(cons['publish date'])

#for simplicity
#only consider annual price
cons=cons[cons['Period Name']=='YEAR']
    
#delete henry hub europe
ind=cons[cons['Location Name']=='EUROPE'][cons['Commodity Product Name']=='HENRY HUB'].index
cons.drop(ind,inplace=True)

#unit conversion
ind=cons[cons['Commodity Product Name'].isin(['ALUMINIUM', 'COPPER', 'NICKEL', 'ZINC'])][cons['Units Name']=='$/LB'].index
cons['Raw Price Value'].loc[ind]=cons['Raw Price Value'].loc[ind]*lbs2ton
ind=cons[cons['Commodity Product Name'].isin(['ALUMINIUM', 'COPPER', 'NICKEL', 'ZINC'])][cons['Units Name']=='C/LB'].index
cons['Raw Price Value'].loc[ind]=cons['Raw Price Value'].loc[ind]*lbs2ton/100

#cleanse
for i in ['Grade Name', 'Price Type Name','Location Name',
 'Product Type Name', 'Value Type Name','Freight Term Name',
 'Purity', 'Forecast Publish Date', 'Currency Name',
 'Nominal Or Real Reference Year','Period Name',
 'First Publish Date', 'Notes', 'Date Of Update']:
    del cons[i]    
cons=cons.sort_values(['Date','publish date',
                  'Commodity Product Name',                   
                  'Units Name','Source Name'])
cons.reset_index(inplace=True,drop=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [4]:
#create output
output=cons[cons['publish date']==pd.to_datetime('2019-08-01')][cons['Date']==pd.to_datetime('2019-12-31')]
output=output.drop_duplicates()
output.reset_index(inplace=True,drop=True)
output.pivot(index='Source Name',columns='Commodity Product Name',
          values='Raw Price Value').to_csv('y0matrix2019.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [5]:
#create output
output=cons[cons['publish date']==pd.to_datetime('2019-08-01')][cons['Date']==pd.to_datetime('2020-12-31')]
output=output.drop_duplicates()
output.reset_index(inplace=True,drop=True)
output.pivot(index='Source Name',columns='Commodity Product Name',
          values='Raw Price Value').to_csv('y1matrix2020.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [6]:
#create output
output=cons[cons['publish date']==pd.to_datetime('2020-08-01')][cons['Date']==pd.to_datetime('2020-12-31')]
output=output.drop_duplicates()
output.reset_index(inplace=True,drop=True)
output.pivot(index='Source Name',columns='Commodity Product Name',
          values='Raw Price Value').to_csv('y0matrix2020.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [7]:
#get spot
spot=pd.read_csv('spot.csv')

#filter commodities
spot=spot[spot['Name'].isin(target_commodity_spot)]

#datetime format
spot['Date']=pd.to_datetime(spot['Date'])

#cleanse
spot.reset_index(inplace=True,drop=True)

In [8]:
#filter year
avg=spot[spot['Date'].dt.year.isin(targetyear)]

#remove na
avg=avg.loc[avg['Value'].dropna().index]

#datetime index
avg.set_index('Date',inplace=True)

#compute annual avg
avg.groupby(['Name']).resample('1A').mean().to_csv('annual.csv')

In [9]:
#compute monthly avg
monthly_avg=avg.groupby(['Name']).resample('1M').mean()

monthly_avg.reset_index(inplace=True)

monthly_avg.pivot(index='Date',columns='Name',values='Value').to_csv('monthly.csv')